In [1]:
from options.options import Options
import os
import torch
from build_dataset_model import build_loaders, build_model
from utils import get_model_attr, calculate_model_losses, tensor_aug
from collections import defaultdict
import math

from torch.utils.tensorboard import SummaryWriter
from tqdm.auto import tqdm
import numpy as np

from new.CustomVAE import *
from utils import calculate_model_losses

In [2]:
args = Options().parse()
if (args.output_dir is not None) and (not os.path.isdir(args.output_dir)):
    os.mkdir(args.output_dir)
if (args.test_dir is not None) and (not os.path.isdir(args.test_dir)):
    os.mkdir(args.test_dir)

# no KL divergence loss
args.use_AE = True

# tensorboard
# writer = SummaryWriter()

# load data
vocab, train_loader, val_loader = build_loaders(args)

dt = train_loader.dataset

| options
dataset: suncg
suncg_train_dir: metadata/data_rot_train.json
suncg_val_dir: metadata/data_rot_val.json
suncg_data_dir: /home/yizhou/Research/SUNCG/suncg_data
loader_num_workers: 8
embedding_dim: 64
gconv_mode: feedforward
gconv_dim: 128
gconv_hidden_dim: 512
gconv_num_layers: 5
mlp_normalization: batch
vec_noise_dim: 0
layout_noise_dim: 32
batch_size: 16
num_iterations: 20000
eval_mode_after: -1
learning_rate: 0.0001
print_every: 100
checkpoint_every: 1000
snapshot_every: 10000
output_dir: ./checkpoints
checkpoint_name: latest_checkpoint
timing: False
multigpu: False
restore_from_checkpoint: False
checkpoint_start_from: None
test_dir: ./layouts_out
gpu_id: 0
KL_loss_weight: 0.1
use_AE: False
decoder_cat: True
train_3d: True
KL_linear_decay: False
use_attr_30: True
manual_seed: 42
batch_gen: False
measure_acc_l1_std: False
heat_map: False
draw_2d: False
draw_3d: False
fine_tune: False
gan_shade: False
blender_path: /home/yizhou/blender-2.92.0-linux64/blender

Starting to read 

In [17]:
args.use_AE = False

In [4]:
for batch in tqdm(train_loader):
    #t += 1
    ids, objs, boxes, triples, angles, attributes, obj_to_img, triple_to_img = tensor_aug(batch)
    z = torch.randn(objs.size(0), 64).to(objs.device)
    break

/home/yizhou/Research/3D_SLN/data/suncg_dataset.py:185: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /opt/conda/conda-bld/pytorch_1607370172916/work/torch/csrc/utils/python_arg_parser.cpp:882.)
  real_objs = (objs != __room__).nonzero().squeeze(1)
/home/yizhou/Research/3D_SLN/data/suncg_dataset.py:185: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /opt/conda/conda-bld/pytorch_1607370172916/work/torch/csrc/utils/python_arg_parser.cpp:882.)
  real_objs = (objs != __room__).nonzero().squeeze(1)
/home/yizhou/Research/3D_SLN/data/suncg_dataset.py:185: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /opt/conda/conda-bl

/home/yizhou/Research/3D_SLN/data/suncg_dataset.py:185: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /opt/conda/conda-bld/pytorch_1607370172916/work/torch/csrc/utils/python_arg_parser.cpp:882.)
  real_objs = (objs != __room__).nonzero().squeeze(1)
/home/yizhou/Research/3D_SLN/data/suncg_dataset.py:185: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /opt/conda/conda-bld/pytorch_1607370172916/work/torch/csrc/utils/python_arg_parser.cpp:882.)
  real_objs = (objs != __room__).nonzero().squeeze(1)
/home/yizhou/Research/3D_SLN/data/suncg_dataset.py:185: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /opt/conda/conda-bl

In [5]:
from torch_geometric.nn import RGCNConv
from torch_scatter import scatter_mean

In [15]:
kwargs = {
        'vocab': dt.vocab,
        'batch_size': args.batch_size,
        'train_3d': args.train_3d,
        'decoder_cat': args.decoder_cat,
        'embedding_dim': args.embedding_dim,
        'gconv_mode': args.gconv_mode,
        'gconv_num_layers': args.gconv_num_layers,
        'mlp_normalization': args.mlp_normalization,
        'vec_noise_dim': args.vec_noise_dim,
        'layout_noise_dim': args.layout_noise_dim,
        'use_AE': args.use_AE
    }

In [18]:
model = RGCNDecoder(**kwargs)

In [19]:
model = model.cuda()

In [20]:
s, p, o = triples.chunk(3, dim=1)  # All have shape (T, 1)
s, p, o = [x.squeeze(1) for x in [s, p, o]]  # Now have shape (T,)
edges = torch.stack([s, o], dim=1)  # Shape is (T, 2)

In [21]:
obj_vecs = model.obj_embeddings_dc(objs)
if model.use_attr:
    attr_vecs = model.attr_embedding_dc(attributes)
    obj_vecs = torch.cat([obj_vecs, attr_vecs], dim=1)
pred_vecs = model.pred_embeddings_dc(p)

In [ ]:
# if self.decoder_cat:
#     obj_vecs = torch.cat([obj_vecs, z], dim=1)
#     #obj_vecs, pred_vecs = self.gconv_net_dc(obj_vecs, pred_vecs, edges)

In [23]:
obj_vecs = torch.cat([obj_vecs, z], dim=1)

In [24]:
obj_vecs.shape

torch.Size([192, 128])

In [25]:
objs.shape

torch.Size([192])

In [36]:
gconv_net_dc = RGCNConv(128, 128, 16).cuda()

In [37]:
edges.shape

torch.Size([374, 2])

In [41]:
output = gconv_net_dc(obj_vecs, edges.transpose(0, 1), edge_type = p)

In [44]:
r = nn.ReLU()

In [46]:
torch.relu(output)

tensor([[2.9697, 0.6766, 1.4268,  ..., 0.3813, 0.0000, 0.0000],
        [1.9221, 0.0000, 1.4203,  ..., 0.0000, 1.3741, 0.0000],
        [0.0000, 0.0000, 1.2423,  ..., 0.7641, 0.3271, 0.0375],
        ...,
        [0.2607, 0.8916, 0.0000,  ..., 2.1181, 0.0000, 0.0000],
        [2.1100, 0.0000, 0.0000,  ..., 3.6803, 0.5734, 0.0000],
        [0.0000, 1.3610, 0.0000,  ..., 0.8145, 0.8190, 0.0636]],
       device='cuda:0', grad_fn=<ReluBackward0>)